In [1]:
import dimod
import neal
import numpy as np
from dwave.system import DWaveSampler, EmbeddingComposite

# Solve binary Linear Program as QUBO with default Solver Advantage_system1.1 

Problem ID: ac847ab8-c03f-4521-bc27-ef2079b13d5f


In [2]:
A = np.array([[0,1,2,3,4,1],
              [2,1,4,1,0,2],
              [1,1,1,1,0,3],
              [1,5,2,3,4,1]])
m, n = A.shape
vartype = dimod.Vartype.BINARY
c = np.array([1,2,1,3,1,4])
b = np.array([1, 3, 2, 6])

# Problem statement:
Given $A\in Z^{mxn}, b\in Z^{m}, c\in Z^{n}$ 

find $x\in F_2^n$ with:

$\min c^T x $ with constraint $Ax=b$ 



This is equivalent to solving QUBO:

$min_x c^Tx + x^T A^T A x - 2 b^TAx + b^Tb$

eventually we need to set weights on the penalty parts $x^T A^T A x - 2 b^TAx + b^Tb$

In [11]:
def printAndGetTestResults(sampler, bqm):
    sampleset = sampler.sample(bqm)
    print("Sampling by ", sampler)
    print(sampleset.to_pandas_dataframe())

In [3]:
# Set up linear and quadratic coefficients
L = c.T - 2 * np.matmul(b.T,A)
Q = np.matmul(A.T,A)
offset = b.T.dot(b)


In [4]:
bqm = dimod.as_bqm(L, Q, offset, vartype)

In [5]:
print(L)
print(bqm.linear)

[-27 -70 -55 -49 -55 -34]
{0: -21.0, 1: -42.0, 2: -30.0, 3: -29.0, 4: -23.0, 5: -19.0}


In [6]:
print(Q + np.diag(L))

[[-21   8  11   6   4   8]
 [  8 -42  17  20  24  11]
 [ 11  17 -30  17  16  15]
 [  6  20  17 -29  24  11]
 [  4  24  16  24 -23   8]
 [  8  11  15  11   8 -19]]


In [8]:
exactSolver = dimod.ExactSolver()
sim = neal.SimulatedAnnealingSampler()
num_reads = 500
exactSet = exactSolver.sample(bqm)
simSet = sim.sample(bqm, num_reads = num_reads)

In [9]:
print(exactSet.first)

Sample(sample={0: 1, 1: 1, 2: 0, 3: 0, 4: 0, 5: 0}, energy=3.0, num_occurrences=1)


In [14]:
print(simSet.to_pandas_dataframe()) # why is num_occurrences not counted up ?

     0  1  2  3  4  5  energy  num_occurrences
0    1  1  0  0  0  0     3.0                1
1    1  1  0  0  0  0     3.0                1
2    1  1  0  0  0  0     3.0                1
3    1  1  0  0  0  0     3.0                1
4    1  1  0  0  0  0     3.0                1
..  .. .. .. .. .. ..     ...              ...
495  1  0  0  1  0  0    12.0                1
496  1  1  0  0  0  0     3.0                1
497  1  1  0  0  0  0     3.0                1
498  1  1  0  0  0  0     3.0                1
499  1  1  0  0  0  0     3.0                1

[500 rows x 8 columns]


In [12]:
sampler = EmbeddingComposite(DWaveSampler())
sampleset = sampler.sample(bqm,num_reads = num_reads)

In [13]:
sampleset.to_pandas_dataframe()

,0,1,2,3,4,5,chain_break_fraction,energy,num_occurrences
0,1,1,0,0,0,0,0.0,3.0,262
1,0,1,0,0,0,0,0.0,8.0,56
2,0,1,0,0,0,1,0.0,11.0,43
3,0,1,1,0,0,0,0.0,12.0,27
4,1,0,0,1,0,0,0.0,12.0,54
5,1,0,0,0,1,0,0.0,14.0,15
6,0,1,0,1,0,0,0.0,19.0,3
7,0,0,1,0,0,0,0.0,20.0,7
8,0,0,0,1,0,0,0.0,21.0,4
9,1,0,1,0,0,0,0.0,21.0,5
